## Explore how a vegetation index is calculated: effect of thresholding

In this illustration, we take as example one RGB image of a single plot (`.png` thumbnail from `drone2report`), and one vegetation index, **GLI** (Green Leaf Index (more [here](https://www.indexdatabase.de/db/i-single.php?id=375)).

We start by import libraries: we are using the `imageio` *Python* library for input/output of image data (png/tiff raster images in this illustration):

In [ ]:
import numpy as np
import numpy.ma as ma
import imageio.v2 as imageio
import matplotlib.pyplot as plt

In [ ]:
fname = '../../paper-drone2report/data/single_rgb_plot/single_rgb_plot.tif'
pic = imageio.imread(fname)

In [ ]:
plt.figure()
plt.imshow(pic)

In [ ]:
print('Shape of the image : {}'.format(pic.shape))

print('Total n. of pixels:', pic.shape[0]*pic.shape[1])

The input image has the characteristics detailed above, e.g. 3 channels and size 707 x 612 (height, width): **total number of input pixels is 432,684**.

## Calculating the GLI index

### Using drone2report

The GLI (Green Leaf Index) was calculated with the `drone2report` software, using [this configuration file: RGB_GLI.ini](https://github.com/ne1s0n/paper-drone2report/blob/main/support_material/RGB_GLI.ini).
The calculations were based on $254\,493$ pixels: 

- average GLI = 0.26837
- median GLI = 0.25397
- std dev GLI = 0.147
- min, max GLU = $[-0.3684, 0.98ì775]$

**<div style="color:red">Question: can GLI be negative?</div>**

### Step-by-step calculations

We need to change the Python environemnt settings so to ignore warnings that are raised when division by zero is encountered: this is something that can easily happen when you work with high dimensional arrays (there may be a zero somewhere):

In [ ]:
## this is a setting to ignore warnings when attempting to divide by 0
np.seterr(divide='ignore', invalid='ignore')

Now, we calculate the GLI index manually:

1. we first define the three channels
2. we then separate the three channels from the `numpy` array:

In [ ]:
channels = ['red','green','blue']
channels.index('red') ## index of the red channel

In [ ]:
red   = pic[:,:,channels.index('red')]
green = pic[:,:,channels.index('green')]
blue  = pic[:,:,channels.index('blue')]

These are three 2D arrays of numbers (pixel intensities), corresponding to the three color channels: we see the zeros that correspond to the black pixels in the four corners around the crop plot (and this shows that we are bound to have divisions by zero):

In [ ]:
red

We then take the equation to calculate the [GLI index](https://www.indexdatabase.de/db/i-single.php?id=375) and implement it using our arrays:

In [ ]:
gli = (2.0*green - red - blue) / (2.0*green + red + blue)
gli[100:120, 220:280]

We see that, due to the presence of manay divisions by zero, `nan` are introduced in the numpy array (missing or undetermined data points): 

In [ ]:
gli

Below, the **heatmap of the GLI values 2D array**: nan are displayed as white (missing values)

In [ ]:
plt.imshow(gli, cmap='summer', interpolation='nearest')
plt.show()

To get the average value of the GLI index for the whole plot (as is done by `drone2report`), we can take the average: the standard `np.mean()` wouldn't work, as there ara `nan`'s in the data; therefore we use a modified version of the function that removes `nan`'s before calculating the average:

In [ ]:
avg = np.nanmean(gli)
print("Average GLI value calculated manually, where divisions by zero returned NaN's:", round(avg,5))

We can force division to zero to return zero instead of `nan`:

In [ ]:
x = (2.0*green - red - blue)
y = (2.0*green + red + blue)

gli = np.divide(x, y, out=np.zeros_like(x), where=y!=0)
gli[100:120, 220:280]

In [ ]:
gli

In the **heatmap of the GLI values 2D array**, now the `nan` are replaced by 0's, i.e. max saturation of those pixels (depending on the chosen color map this will be displayed as the darkest possible color in that channel)

In [ ]:
plt.imshow(gli, cmap='summer', interpolation='nearest')
plt.show()

Now there are no `nan`'s, therefore we can use the usual function `np.mean()`: however, 0's are numbers!! 
They add nothing to the numerator, but add counts to the denominator, and the results is consequently biased downward:

In [ ]:
avg = np.mean(gli)
print("Average GLI value calculated manually, where divisions by zero returned 0's:", round(avg,5))

We see that the corners around the crop plot introduce some operational and interpretation problems, either when they are `nan`'s or when they're forced to be 0's.

A better solution is needed: this is **masking**, i.e. unnecessary pixels like those in the corners are "masked", hence not used in the calculations (no problems with `nan`s or 0's).

Before moving on to masking, we wrap all the code needed to calculate the GLI index into a **function**:

In [ ]:
def GLI(img, channels):
	"""Green leaf index, uses red, green, blue"""
	try:
		red   = img[:,:,channels.index('red')]
		green = img[:,:,channels.index('green')]
		blue  = img[:,:,channels.index('blue')]
	except ValueError:
		#if this clause is activated it means that the requested channel(s) are not available
		return np.nan
	#if we get here the index can be applied to the current image
	return(
		(2.0*green - red - blue) / 
		(2.0*green + red + blue)
	) 

In [ ]:
gli = GLI(pic, channels)

In [ ]:
gli[100:120, 220:280]

In [ ]:
np.nanmean(gli)

## Masked arrays

### Masking unnecessary values in the index matrix

We first try masking unnecessary values of the index (the triangular margins outside of the ROI).
We do this by **masking index values calculated on out-of-shape pixels**.

This is a quick-n-dirty initial attempt.
A better way would be to mask directly the grey pixels in the input RGB image data (TODO)

In [ ]:
gli

In [ ]:
mask_0 = np.isnan(gli)

In [ ]:
mask_0

In [ ]:
masked_gli = ma.masked_array(gli, mask_0)

In [ ]:
masked_gli

In [ ]:
masked_pixels = np.ma.sum(mask_0)
print(masked_pixels)

The total n. of masked pixels is $185\,975$

The total n. of pixels is given by the product of the width and height of the array:

In [ ]:
mask_0.shape[0] * mask_0.shape[1]

We can now calculate the number of pixels used for the calculation of the average index:

In [ ]:
(mask_0.shape[0] * mask_0.shape[1]) - np.ma.sum(mask_0)

$254\,486$ pixels were used to claculate the average GLI index from the matrix of GLI values per pixel.

**<div style="color:red">Check: this is slightly different from the number of used pixels reported by `drone2report`: $254\,493$ pixels</div>**

#### Average GLI value on single plot calculated after masking

Masking was based on GLI values (0s were masked):

In [ ]:
round(np.ma.mean(masked_gli),5)

### Masking directly the input data

An even better approach, at least in principle, is to mask the unnecessary pixels directly in the image data, before calculating the index:

In [ ]:
pic = imageio.imread(fname)

Sanity check: let's view the current image

In [ ]:
plt.figure()
plt.imshow(pic)

**Important**: we have three channels (RGB), and the corners in the `tif` image are black, hence the corresponding pixels are zero-valued in all three channels:

In [ ]:
pic[:,:,2]

For the masking, we need to impose the condition across layers: we use `np.all()` on the second axis, as we want the condition to be true if the same pixel position is zero in all three channels:

In [ ]:
mask_0 = np.all(pic == 0, axis=2)

We check the result visually with a heatmap: 

In [ ]:
plt.imshow(mask_0, cmap='grey', interpolation='nearest')
plt.show()

In [ ]:
mask_0[100:160,160:220]

The n. of masked pixels is slightly different compared to the masking of the GLI index values:

In [ ]:
np.ma.sum(mask_0)

In [ ]:
mask_0.shape

In [ ]:
mask_tot = np.dstack((mask_0,mask_0,mask_0))
mask_tot.shape

In [ ]:
mask_tot[100:160,160:220,0]

In [ ]:
pic[120:160, 160:220, 2]

In [ ]:
masked_pic = ma.masked_array(pic, mask_tot)

In [ ]:
masked_pic[120:160, 180:220, 0]

In [ ]:
red   = masked_pic[:,:,channels.index('red')]
green = masked_pic[:,:,channels.index('green')]
blue  = masked_pic[:,:,channels.index('blue')]

In [ ]:
red[120:160, 180:220]

In [ ]:
temp = (2.0*green - red - blue) / (2.0*green + red + blue)
temp[120:160, 180:220]

In [ ]:
round(np.ma.mean(temp),5)

In [ ]:
temp

In [ ]:
temp.shape

In [ ]:
x = temp[temp.mask == False]

In [ ]:
x

In [ ]:
x.shape

In [ ]:
plt.hist(x, bins=60)
plt.show() 